In [47]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import folium
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMap

import utm 

import requests
import json


In [21]:
%run functions.ipynb

In [71]:
def Generate_basemap():
    basemap = folium.Map(location=[40.730610 , -73.935242])
    return basemap

# 1 Data Preparation

#### 1.1 List of Stations (Lat,Lon,Capacity)

In [ ]:
# URL of the JSON file
url = 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_information.json'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    json_data = response.json()
    
    # Now you can work with the JSON data
    print(json_data)
else:
    print('Failed to retrieve data:', response.status_code)

# Create a DataFrame from the 'stations' list
station_data = pd.DataFrame(json_data['data']['stations'])

# Select only the required columns
station_data = station_data[['short_name', 'name', 'region_id', 'lat', 'lon', 'capacity']]

In [74]:
station_data.iloc[:3]

,short_name,name,region_id,lat,lon,capacity
0,6215.04,W 25 St & 6 Ave,71,40.743954,-73.991449,51
1,5216.06,Vesey St & Church St,71,40.712220,-74.010472,48
2,6621.06,31 Ave & 57 St,71,40.757357,-73.904726,23


##### 1.1.1 Station Visualisation

In [72]:
basemap = Generate_basemap()
FastMarkerCluster(station_data[['lat', 'lon' , 'capacity']]).add_to(basemap)

HeatMap(station_data[['lat', 'lon' , 'capacity']]).add_to(basemap)
basemap

In [79]:
# Sample data
locations = station_data['name']
latitudes = station_data['lat']
longitudes = station_data['lon']
sizes = station_data['capacity']
#values = [20, 30, 25, 40]  # Values for the color scale

# Initialize the map centered around the first location
mymap = folium.Map()

# Iterate over locations
for lat, lon, size, location in zip(latitudes, longitudes, sizes, locations):
    # Add circle marker with varying sizes
    folium.CircleMarker(
        location=[lat, lon],
        radius=size / 10,  # Normalize size for better visualization
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=location
    ).add_to(mymap)

# Save the map to an HTML file
mymap.save("nyc_bike_stations.html")

#### 1.2 Ride Data

In [62]:
# Open all files / computing intensive
base_folder_path = 'C:\\Users\\lukas\\OneDrive - Imperial College London\\0_Data\\nyc_bike_rental\\data'
start_year = 2015
end_year = 2024
# combined_data = combine_csv_files_in_years(base_folder_path,start_year,end_year)

In [63]:
combined_data.describe()

,Trip Duration,Start Station ID,Start Station Latitude,Start Station Longitude,End Station ID,End Station Latitude,End Station Longitude,Bike ID,Birth Year,Gender,...,end station id,end station latitude,end station longitude,bikeid,birth year,gender,start_lat,start_lng,end_lat,end_lng
count,3.396200e+05,339620.000000,339620.000000,339620.000000,339620.000000,339620.000000,339620.000000,339620.000000,313288.000000,339620.000000,...,1.329984e+06,1.329984e+06,1.329984e+06,1.329984e+06,1.312074e+06,1.329984e+06,2.479054e+06,2.479054e+06,2.474097e+06,2.474097e+06
mean,9.629629e+02,3206.818214,40.723126,-74.046422,3203.450032,40.722469,-74.045612,24911.866754,1979.186748,1.122720,...,3.272430e+03,4.069809e+01,-7.400202e+01,3.198455e+04,1.980866e+03,1.137917e+00,4.073170e+01,-7.404052e+01,4.073164e+01,-7.404025e+01
std,4.868570e+04,26.610566,0.008139,0.011194,62.608460,0.070325,0.127556,743.745408,9.652899,0.519344,...,1.656609e+02,9.896964e-01,1.799568e+00,6.323833e+03,1.048134e+01,5.445743e-01,1.216913e-02,1.213060e-02,1.229517e-02,1.212820e-02
min,6.100000e+01,3183.000000,40.692640,-74.096937,147.000000,0.000000,-74.096937,14552.000000,1900.000000,0.000000,...,7.900000e+01,0.000000e+00,-7.409694e+01,1.452900e+04,1.887000e+03,0.000000e+00,4.067833e+01,-7.408896e+01,4.064000e+01,-7.419000e+01
25%,2.470000e+02,3186.000000,40.717732,-74.050656,3186.000000,40.716540,-74.050444,24486.000000,1974.000000,1.000000,...,3.191000e+03,4.071773e+01,-7.405039e+01,2.919800e+04,1.973000e+03,1.000000e+00,4.071980e+01,-7.404629e+01,4.071959e+01,-7.404595e+01
50%,3.840000e+02,3202.000000,40.721525,-74.044247,3199.000000,40.721124,-74.043845,24602.000000,1981.000000,1.000000,...,3.205000e+03,4.071959e+01,-7.404312e+01,2.953200e+04,1.983000e+03,1.000000e+00,4.073117e+01,-7.403798e+01,4.073101e+01,-7.403798e+01
75%,6.560000e+02,3211.000000,40.727596,-74.038051,3211.000000,40.727224,-74.036486,24711.000000,1986.000000,1.000000,...,3.273000e+03,4.072722e+01,-7.403805e+01,3.361200e+04,1.989000e+03,1.000000e+00,4.074097e+01,-7.403160e+01,4.074097e+01,-7.403160e+01
max,2.026021e+07,3426.000000,40.752559,-74.032108,3442.000000,40.801343,0.000000,29296.000000,2000.000000,2.000000,...,4.071000e+03,4.084828e+01,0.000000e+00,4.998500e+04,2.004000e+03,2.000000e+00,4.086394e+01,-7.394117e+01,4.087241e+01,-7.388827e+01


In [64]:
# Open only 2024
folder_path = 'C:\\Users\\lukas\\OneDrive - Imperial College London\\0_Data\\nyc_bike_rental\\data\\2024'
data_2024 = combine_csv_files(folder_path)
data_2024['started_at'] = pd.to_datetime(data_2024['started_at'])

In [67]:
# Group by start_station_id and started_at, and calculate count, start_lat, and start_lng for each group
grouped_data = data_2024.groupby(['start_station_name', 'started_at']).agg({
    'start_lat': 'first',
    'start_lng': 'first',
    'ride_id': 'count'  # Count the number of objects standing at each station at each time point
}).reset_index()

# Rename the 'ride_id' column to 'count' for clarity
grouped_data.rename(columns={'ride_id': 'count'}, inplace=True)

# Find the maximum count of objects standing at any given time for each station
max_objects_count = grouped_data.groupby('start_station_name')['count'].max()
max_objects_count

start_station_name
11 St & Washington St                2
12 St & Sinatra Dr N                 1
14 St Ferry - 14 St & Shipyard Ln    2
2 St & Park Ave                      2
4 St & Grand St                      1
                                    ..
Van Vorst Park                       2
Warren St                            2
Washington St                        1
Willow Ave & 12 St                   1
York St & Marin Blvd                 2
Name: count, Length: 89, dtype: int64

# 2 Data Analysis